In [ ]:
import os
import platform
import shutil
import numpy
from pathlib import Path
from numpy import size  

In [ ]:
#function to check your os and zip the file 
def zipFolder(pathtoSubjectsFolder, your_os): 
    
    path = Path(pathtoSubjectsFolder)
    parent = str(path.parent.absolute())
    folder = os.path.basename(path)
    parent
    folder

    if "macOS" in your_os or "linux" in your_os.lower(): 
        zip_string="zip -r zippedData.zip {foldername} -x '**/.DS_Store'".format(foldername = folder)
        changeDirectory = "cd {parentDirectory}".format(parentDirectory  = parent)
        os.system(changeDirectory)   
        os.system(zip_string)    
    elif "windows" in your_os.lower():
        print("Please zip data using right click to zip folder (the typical method to zip folders on windows)")
    else:
        print("os match not found cannot zip")
        exit() 

    return parent

def getFolderSize(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size/1e+9


def validator(pathToData, pathToDeafultParams):
    #function for validation
    errors = 0
    for dirs in os.walk(pathToData):
        dirpath = dirs[0]
        dirpath = dirpath.replace("\\", "/")

        # #Verfiy no sub/sess/scan name is repeated: 
        # # folders = os.listdir(dirpath)
        # folders = [f for f in os.listdir(dirpath) if not os.path.isfile(os.path.join(dirpath, f))]
        # print(folders)

        files = [f for f in os.listdir(dirpath) if os.path.isfile(os.path.join(dirpath, f))]
        if '.DS_Store' in files:
            files.remove('.DS_Store')
        if size(files) > 0:

            #verify there are no files in the wrong place
            if 'nirs' not in dirpath:
                print("There is an error in your file structure there are file(s) present in folder, ", dirpath, " that should not be there")
            if 'nirs' in dirpath:
                dirpathComponents = dirpath.split("/")
                nirsfolder = dirpathComponents[-1]
                scanfolder = dirpathComponents[-2]
                sessionfolder = dirpathComponents[-3]
                subjectfolder = dirpathComponents[-4]
                
                if 'nirs' not in nirsfolder:
                    print("The nirs folder is not named correctly at: ", dirpath, " please make sure folder name is 'nirs' OR folder structure is incorrect")
                    errors +=1
                if 'scan-' not in scanfolder:
                    print("The scan folder is not named correctly at: ", dirpath, " please make sure folder name is 'scan-####' where ## are unique numbers as ids OR folder structure is incorrect")
                    errors +=1
                if 'ses-' not in sessionfolder:
                    print("The session folder is not named correctly at: ", dirpath, " please make sure folder name is 'ses-####' where ## are unique numbers as ids OR folder structure is incorrect")
                    errors +=1
                if 'sub-' not in subjectfolder:
                    print("The subject folder is not named correctly at: ", dirpath, " please make sure folder name is 'sub-####' where ## are unique numbers as ids OR folder structure is incorrect")
                    errors +=1

                files = [f for f in os.listdir(dirpath) if os.path.isfile(os.path.join(dirpath, f))]
                if '.DS_Store' in files:
                    files.remove('.DS_Store')
                # print(files)
                paramsPresent = [True for f in files if 'params' in f]
                if True not in paramsPresent:
                    if pathToDeafultParams == "": 
                        print("params.txt is missing for scan: ", dirpath)
                    else:
                        print("Copying default params from: ", pathToDeafultParams, " to ", dirpath)
                        shutil.copy(pathToDeafultParams, dirpath)
                else:
                    files.remove('params.txt')

                if '.snirf' not in files[0] and '.nirs' not in files[0] and '.mat' not in files[0]: 
                    print("Scan folder does not have nirs/snirf/ndot files: ", dirpath)

    return errors


In [ ]:
#Thigs to edit: 
pathToData = "/path/to/subjects/folder"

#If you provide a default Params that file will be copied into any subject nirs folder that does not have one if you do not want to copy a params accross scans make it = ""
pathToDeafultParams="/path/to/params.txt"

#you should only edit your_os if the matcher is not working and you get the error statement "os match not found cannot zip"
your_os = platform.platform()

#DO NOT EDIT ANYTHING BELOW THIS LINE
if ' ' in pathToData or ' ' in pathToDeafultParams:
    print("Please remove any ' ' chars (space) in paths to data or params files before proceeding")
    exit()
else:
    foldersize = getFolderSize(pathToData)
    if foldersize > 4.0:
        print("WARNING FILES GREATER THAN 4.0 GIGS HAVE HIGH RISK OF CORRUPTION DURING CORRUPTION AND AS A RESULT WILL NOT UPLOAD TO OXI IT IS RECOMMENDED YOU CHUNK MULTIPLE SUBJECTS INTO FOLDER SIZES LESS THAN 4.0 GB")
        print("Would you still like to procede with validation and compression?")
        confirm = ""
        while confirm != "yes": 
            print("Please type 'yes' to continue")
            confirm = input()
    #validate paths
    errors = 0 
    errors = validator(pathToData, pathToDeafultParams)

    #zipdirectory if no errors 
    if errors>0: 
        print("please fix the", errors, "errors before attempting to zip")
    else:
        parentDir = zipFolder(pathToData, your_os)
        if "macOS" in your_os or "linux" in your_os.lower(): 
            print("Done zipping folder available at", parentDir, "called zippedData.zip")
        elif "windows" in your_os.lower():
            print("You are ready to zip your file for upload to OXI")